In [15]:

import pandas as pd

In [16]:
%store -r rvu


In [18]:
rvu.head(5)

,State,PatientNumber,CptCode,CptDesc,CptStatus,ProviderNumber,ProviderSpecialty,PlaceofService,PayerName,Transactions,Units,UnitsCharge,UniquePatients,Charge,Payment,Adjustment,Net
0,Alaska,AL83985207,3077F,Syst bp >/= 140 mm hg,Not Valid for Medicare,Provider 1073,Internal Medicine,Office,Commercial,Insurance Payment,0,0,1,0.0,0.0,0.0,0.0
1,Alaska,AL83985207,3077F,Syst bp >/= 140 mm hg,Not Valid for Medicare,Provider 1073,Internal Medicine,Office,Commercial,Blood pressure,1,0,0,0.0,0.0,0.0,0.0
2,Alaska,AL83985207,3077F,Syst bp >/= 140 mm hg,Not Valid for Medicare,Provider 1073,Internal Medicine,Office,Commercial,Bill,0,0,0,0.0,0.0,0.0,0.0
3,Alaska,AL83985207,3077F,Syst bp >/= 140 mm hg,Not Valid for Medicare,Provider 1073,Internal Medicine,Office,Commercial,Invoice - Transfer,0,0,0,0.0,0.0,0.0,0.0
4,Alaska,AL83985207,3079F,Diast bp 80-89 mm hg,Not Valid for Medicare,Provider 1073,Internal Medicine,Office,Commercial,Insurance Payment,0,0,0,0.0,0.0,0.0,0.0


# Analyze RVU

## Payer RVU: Part 1
<p>
This analysis that I did here, is based on Payer, as you can see, our actual volume of Medicaid is lower than commercial and Medicare. I pulled in our charges and payments, and I calculated a payment per unit. So, Medicare is 100% Medicare, which means this payment per unit for a commercial payer is 119% of Medicare and Medicaid is 61%. I expected Medicaid to be a little higher, although you expect Medicaid to be lower than Medicare. This is because I pulled a small volume of a fake medical dataset, and I don't have a lot of Medicaid built into this data set, so it might be a little bit wide. However, in the real world you would see Medicaid being less than Medicare. Then I took a gross collection rate. Gross collection rate is just your charges. This calculation doesn't really tell us too much because it's just a gross collection rate, but our commercial gross collection rate leads at 51%.
</p>

In [20]:
rpt1 = rvu.groupby("PayerName").apply(
    lambda d: pd.Series(
        {
            "Sum of Units w/Charge": d["UnitsCharge"].sum(),
            "Sum of Charges": d["Charge"].sum(),
            "Sum of Payment": d["Payment"].sum() * -1,
            "Pmt/Unit":d["Payment"].sum() / d["UnitsCharge"].sum() * -1,
       
            "Gross Collection Rate":d["Payment"].sum() / d["Charge"].sum() * -100
        }
    ))
rpt1["% of Medicare"]= rpt1['Pmt/Unit']/ rpt1.loc['Medicare',"Pmt/Unit"] * 100
rpt1.loc['Total'] = pd.Series(rpt1[["Sum of Units w/Charge", "Sum of Charges", "Sum of Payment", "Pmt/Unit", "% of Medicare"]]
                   .sum(), index = ["Sum of Units w/Charge", "Sum of Charges", "Sum of Payment", "Pmt/Unit"])
rpt1.style.format({"Sum of Units w/Charge": "${:20,.0f}", 
                          "Sum of Charges": "${:20,.0f}", 
                          "Sum of Payment": "${:20,.0f}",
                          "Pmt/Unit":"${:20,.0f}",
                          "% of Medicare": "{:20,.0f}%",
                          "Gross Collection Rate":"${:20,.0f}%"
                 }).background_gradient(cmap='Blues')

,Sum of Units w/Charge,Sum of Charges,Sum of Payment,Pmt/Unit,Gross Collection Rate,% of Medicare
PayerName,,,,,,
Commercial,"$ 6,702","$ 1,112,663","$ 562,290",$ 84,$ 51%,119%
Medicaid,$ 524,"$ 96,514","$ 22,356",$ 43,$ 23%,61%
Medicare,"$ 7,124","$ 1,258,353","$ 500,227",$ 70,$ 40%,100%
Total,"$ 14,350","$ 2,467,531","$ 1,084,874",$ 197,$ nan%,nan%


## Payer RVU: Part 2

<p>
I followed the exact same report above, but I wanted to understand our debt collection ratio, which means I needed to understand what our contracts were. I pulled in our adjustments for contractual and I took them less are charges which gave me net charges. Then I was able to calculate our net collection rate for each pair. Again, I expect these to be higher. Normally you would expect the collection rate for Commercial to be higher than Medicare, and in the real world the collection rate percentage would gradually go up as time goes.
</p>

In [21]:
rpt2 = rvu.groupby("PayerName").apply(
    lambda d: pd.Series(
        {
            "Sum of Adjustment": d["Adjustment"].sum() * -1,
            "Sum of Charges": d["Charge"].sum(),
            "Sum of Payment": d["Payment"].sum() * - 1,
            "Net Charges":d["Charge"].sum()  + d["Adjustment"].sum(),
            "Net Collection Rate":d["Payment"].sum() / (d["Charge"].sum()  + d["Adjustment"].sum())* -100
        }
    ))
rpt2.loc['Total'] = pd.Series(rpt2[["Sum of Adjustment", "Sum of Charges", "Sum of Payment", "Net Charges"]]
                  .sum(), index = ["Sum of Adjustment", "Sum of Charges", "Sum of Payment", "Net Charges"])
rpt2.style.format({"Sum of Adjustment": "${:20,.0f}", 
                          "Sum of Charges": "${:20,.0f}", 
                          "Sum of Payment": "${:20,.0f}",
                          "Net Charges":"${:20,.0f}",
                          "Net Collection Rate":"${:20,.0f}%"
                 }).background_gradient(cmap='Blues')

,Sum of Adjustment,Sum of Charges,Sum of Payment,Net Charges,Net Collection Rate
PayerName,,,,,
Commercial,"$ 431,553","$ 1,112,663","$ 562,290","$ 681,110",$ 83%
Medicaid,"$ 56,946","$ 96,514","$ 22,356","$ 39,568",$ 57%
Medicare,"$ 666,774","$ 1,258,353","$ 500,227","$ 591,580",$ 85%
Total,"$ 1,155,273","$ 2,467,531","$ 1,084,874","$ 1,312,257",$ nan%


## Collection Rate for Provider
<p>You can see the different collection rates for each one of our specialties.  In fact, I would be curious, like what is cardiology, which is 32%. I would expect that to be more in the range of 90 to 95%. That's where you want your entire collection to be. If your collection rate is this low this shows that the revenue cycle needs help, and I would start at the Front-End-Process. for instance, Patient registration, Insurance verification and Preauthorization 
</p>

In [22]:
rpt3 = rvu.groupby("ProviderSpecialty").apply(
    lambda d: pd.Series(
        {
            "Sum of Adjustment": d["Adjustment"].sum() * -1,
            "Sum of Charge": d["Charge"].sum(),
            "Sum of Payment": d["Payment"].sum() * -1,
            "Net Charges":d["Charge"].sum() + d["Adjustment"].sum(),
            "Net Collection Rate":d["Payment"].sum() / d["Charge"].sum() * -100
        }
    ))
rpt3.loc['Total'] = pd.Series(rpt3[["Sum of Adjustment", 'Sum of Charge', "Sum of Payment", "Net Charges"]]
                              .sum(), index = ["Sum of Adjustment", 'Sum of Charge', "Sum of Payment", "Net Charges"])
rpt3.style.format({"Sum of Adjustment": "${:20,.0f}", 
                          "Sum of Charge": "${:20,.0f}", 
                          "Sum of Payment": "${:20,.0f}",
                          "Net Charges":"${:20,.0f}",
                          "Net Collection Rate":"{:20,.0f}%"}).background_gradient(cmap='Greens')

,Sum of Adjustment,Sum of Charge,Sum of Payment,Net Charges,Net Collection Rate
ProviderSpecialty,,,,,
Allergy,$ 523,"$ 1,607",$ 858,"$ 1,084",53%
Cardiology,"$ 308,577","$ 494,042","$ 157,811","$ 185,465",32%
Emergency Medicine,"$ 73,367","$ 173,699","$ 88,819","$ 100,332",51%
Endocrinology,"$ 1,455","$ 5,744","$ 2,582","$ 4,289",45%
Family Medicine,"$ 73,987","$ 240,285","$ 148,367","$ 166,298",62%
Gastroenterology,"$ 8,293","$ 19,293","$ 9,091","$ 11,000",47%
Geriatrics,"$ 4,437","$ 13,183","$ 6,587","$ 8,746",50%
Hematology/Oncology,"$ 6,202","$ 17,454","$ 9,698","$ 11,252",56%
Hospice,"$ 1,058","$ 3,892","$ 2,314","$ 2,833",59%


## Collection Rate For Place of service
<p>
This calculation will show you your average charges per unique patient, which is kind of cool.  This is done by Medicare place of service. Unique patients mean that if a patient comes in 10 times, they're a unique patient once. This analysis can help you understand what you're charging and what you're receiving per unique patient. This is very important because you might have patients who come in frequently who you're charging, but you're not clicking on, or claims aren't configured correctly in your EMR system. For example, this place of service Home, as a poor or a worse gross collection rate at 27%. This should be relatively easy to figure because the place service Home only collected 16 unique patients. However more importantly the solution may positively impact other places of service, especially if it’s a Front-End-Process issue. 
</p>

In [23]:
rpt4 = rvu.groupby("PlaceofService").apply(
    lambda d: pd.Series(
        {
            "Unique Patient": d["UniquePatients"].sum(),
            "Net Charge": d["Charge"].sum() ,
            "Payment": d["Payment"].sum() * -1 ,
            "Avg. Chg/Pt":d["Charge"].sum() / d["UniquePatients"].sum(),
            "Avg. Pmt/Pt":d["Payment"].sum() / d["UniquePatients"].sum() * - 1,
            "GCR/Patient": ((d["Payment"].sum() / d["UniquePatients"].sum()) / (d["Charge"].sum() / d["UniquePatients"].sum()) * - 100)
        }
    ))
rpt4.loc['Total'] = pd.Series(rpt4[['Unique Patient', 'Net Charge', 'Payment', 'Avg. Chg/Pt', 'Avg. Pmt/Pt','GCR/Patient']]
                              .sum(), index = ['Unique Patient', 'Net Charge', 'Payment'])
rpt4.style.format({"Unique Patient": "{:20,.0f}", 
                          "Net Charge": "${:20,.0f}", 
                          "Payment": "${:20,.0f}",
                          "Avg. Chg/Pt":"${:20,.0f}",
                          "Avg. Pmt/Pt":"${:20,.0f}",
                          "GCR/Patient":"{:20,.0f}%"}).background_gradient(cmap='Greens')

,Unique Patient,Net Charge,Payment,Avg. Chg/Pt,Avg. Pmt/Pt,GCR/Patient
PlaceofService,,,,,,
Ambulatory surgical center,10,"$ 9,742","$ 4,878",$ 974,$ 488,50%
Emergency room - hospital,556,"$ 276,656","$ 126,408",$ 498,$ 227,46%
Home,16,"$ 6,529","$ 1,739",$ 408,$ 109,27%
Inpatient hospital,"1,294","$ 505,424","$ 220,817",$ 391,$ 171,44%
Nursing facility,14,"$ 2,572","$ 1,295",$ 184,$ 92,50%
Off Campus-Outpatient Hospital,712,"$ 205,860","$ 63,368",$ 289,$ 89,31%
Office,"3,000","$ 722,277","$ 410,260",$ 241,$ 137,57%
On Campus-Outpatient Hospital,"2,506","$ 727,707","$ 251,066",$ 290,$ 100,35%
Skilled nursing facility,58,"$ 10,764","$ 5,041",$ 186,$ 87,47%


## Outpatient Visit est

In [25]:
values_list = ['99217','99219']
rpt4 =  rvu[(rvu['CptCode'].str.contains('9921')) & (~health['CptCode'].isin(values_list))]
rpt4 = rpt4.groupby("CptCode").apply(
    lambda d: pd.Series(
        {
            
            "Sum of Charge": d["Charge"].sum() ,
            "Sum of Payment": d["Payment"].sum() * -1 ,
            "Sum of Units":d["Units"].sum()
           
          
        }
    ))
rpt4.loc['Total'] = pd.Series(rpt4[["Sum of Charge", "Sum of Payment", "Sum of Units"]]
                   .sum(), index = ["Sum of Charge", "Sum of Payment", "Sum of Units"])
rpt4.style.format({"Sum of Charge": "{:20,.0f}", 
                          "Sum of Payment": "${:20,.0f}", 
                          "Sum of Units": "${:20,.0f}"}).format_index(str.upper, axis=1).background_gradient(cmap='Greens')

,SUM OF CHARGE,SUM OF PAYMENT,SUM OF UNITS
CptCode,,,
99211,463,$ 149,$ 14
99212,"4,832","$ 2,392",$ 66
99213,"131,638","$ 74,514","$ 1,102"
99214,"260,647","$ 151,200","$ 1,484"
99215,"34,700","$ 19,860",$ 150
Total,"432,281","$ 248,115","$ 2,816"


In [26]:
rvu.head()#

,State,PatientNumber,CptCode,CptDesc,CptStatus,ProviderNumber,ProviderSpecialty,PlaceofService,PayerName,Transactions,Units,UnitsCharge,UniquePatients,Charge,Payment,Adjustment,Net
0,Alaska,AL83985207,3077F,Syst bp >/= 140 mm hg,Not Valid for Medicare,Provider 1073,Internal Medicine,Office,Commercial,Insurance Payment,0,0,1,0.0,0.0,0.0,0.0
1,Alaska,AL83985207,3077F,Syst bp >/= 140 mm hg,Not Valid for Medicare,Provider 1073,Internal Medicine,Office,Commercial,Blood pressure,1,0,0,0.0,0.0,0.0,0.0
2,Alaska,AL83985207,3077F,Syst bp >/= 140 mm hg,Not Valid for Medicare,Provider 1073,Internal Medicine,Office,Commercial,Bill,0,0,0,0.0,0.0,0.0,0.0
3,Alaska,AL83985207,3077F,Syst bp >/= 140 mm hg,Not Valid for Medicare,Provider 1073,Internal Medicine,Office,Commercial,Invoice - Transfer,0,0,0,0.0,0.0,0.0,0.0
4,Alaska,AL83985207,3079F,Diast bp 80-89 mm hg,Not Valid for Medicare,Provider 1073,Internal Medicine,Office,Commercial,Insurance Payment,0,0,0,0.0,0.0,0.0,0.0


In [116]:
rpt5 =  rvu[rvu['Transactions'].str.contains('adjustment')]
rpt5 = rpt5.groupby(['ProviderSpecialty']).apply(
    lambda d: pd.Series(
        {
            
            "Sum of Adjustment": d["Adjustment"].sum() * -1,
            'Providers Count': d['ProviderNumber'].value_counts().sum()
          
        }
    )).sort_values(by='Sum of Adjustment', ascending=False)
rpt5.loc['Total'] = pd.Series(rpt5[["Sum of Adjustment",'Providers Count']]
                   .sum(), index = ["Sum of Adjustment",'Providers Count'])
rpt5.style.format({"Sum of Adjustment": "${:20,.3f}",
                   'Providers Count':"{:20,.0f}"
                  }).format_index(str.upper, axis=1).background_gradient(cmap='Greens')

,SUM OF ADJUSTMENT,PROVIDERS COUNT
ProviderSpecialty,,
Emergency Medicine,"$ 1,955.938",66
Cardiology,"$ 1,583.048",166
Radiology,"$ 1,297.056",174
OB/GYN,"$ 1,089.805",24
Internal Medicine,"$ 1,048.843",50
Pediatrics,$ 953.002,24
Family Medicine,$ 800.936,46
Hospitalist,$ 599.917,24
Infectious Disease,$ 508.085,18


In [126]:
rpt5_test = rvu[rvu['Transactions'].str.contains('adjustment')].groupby(['ProviderSpecialty','ProviderNumber']).apply (
 lambda d: pd.Series(
        {
            
            "Sum of Adjustment": d["Adjustment"].sum() * -1,
            'Providers Count': d['ProviderNumber'].value_counts().sum()
          
        }
    )).sort_values(by='Sum of Adjustment', ascending=False)

rpt5_test.loc['Total'] = pd.Series(rpt5_test[["Sum of Adjustment",'Providers Count']]
                   .sum(), index = ["Sum of Adjustment",'Providers Count'])
rpt5_test.style.format({"Sum of Adjustment": "${:20,.3f}",
                        'Providers Count':"{:20,.0f}"}).format_index(str.upper, axis=1).background_gradient(cmap='Greens')

,SUM OF ADJUSTMENT,PROVIDERS COUNT
"('OB/GYN', 'Provider 795')",$ 466.424,2
"('Pediatrics', 'Provider 888')",$ 400.636,2
"('OB/GYN', 'Provider 1070')",$ 280.051,4
"('Emergency Medicine', 'Provider 133')",$ 275.998,8
"('Emergency Medicine', 'Provider 345')",$ 275.728,10
"('Infectious Disease', 'Provider 992')",$ 217.699,8
"('Emergency Medicine', 'Provider 588')",$ 213.494,8
"('Hospitalist', 'Provider 146')",$ 188.352,8
"('Nephrology', 'Provider 615')",$ 180.476,6
"('Radiology', 'Provider 244')",$ 175.118,24


In [ ]:
rvu